# 🐔 Caduceus 鸡基因组预训练数据准备 (GRCg6a)

此 notebook 用于下载和预处理 **GRCg6a** 鸡基因组数据，供 H200 服务器离线训练使用。

**为什么选择 GRCg6a？**
- 与 MPRA 实验数据坐标一致
- ChickenGTEx 数据可通过 LiftOver 转换
- 避免复杂的坐标转换问题

**运行步骤：**
1. 依次运行所有 cell
2. 最后会自动下载 `chicken_pretrain_data_GRCg6a.tar.gz`
3. 将下载的文件上传到 H200 服务器

**预计时间：** 15-20 分钟（取决于网络速度）

In [17]:
# Cell 1: 安装依赖
!pip install -q biopython tqdm h5py numpy

In [18]:
# Cell 2: 导入库和配置
import os
import gzip
import random
import json
import tarfile
import urllib.request
import pickle
import h5py
import numpy as np
from pathlib import Path
from tqdm import tqdm
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict

# 目录配置
DATA_DIR = Path('/content/chicken_genome')
PROCESSED_DIR = Path('/content/processed')
OUTPUT_DIR = Path('/content/output')

DATA_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ============================================
# GRCg6a 基因组下载链接 (galGal6)
# ============================================
GENOME_URL = 'https://ftp.ensembl.org/pub/release-104/fasta/gallus_gallus/dna/Gallus_gallus.GRCg6a.dna.toplevel.fa.gz'
GENOME_URL_BACKUP = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/002/315/GCF_000002315.6_GRCg6a/GCF_000002315.6_GRCg6a_genomic.fna.gz'

# ============================================
# 预处理参数 - 针对 Caduceus 65k 预训练优化
# ============================================
CONFIG = {
    # 序列长度参数
    'seq_length': 65536,           # 65k bp，Caduceus 训练序列长度
    'stride': 32768,               # 50% overlap，增加数据量
    'min_seq_length': 10000,       # 最小序列长度

    # 质量过滤
    'max_n_ratio': 0.05,           # 最大 N 比例 5%
    'min_gc_ratio': 0.30,          # 最小 GC 含量
    'max_gc_ratio': 0.70,          # 最大 GC 含量

    # 数据集分割
    'val_ratio': 0.02,             # 验证集比例
    'test_ratio': 0.01,            # 测试集比例

    # 染色体过滤 (只使用主要染色体)
    'use_main_chromosomes': True,
    'main_chromosomes': [str(i) for i in range(1, 34)] + ['W', 'Z', 'MT'],

    # 随机种子
    'seed': 42,
}

print("="*60)
print("Chicken Caduceus 预训练数据准备 (GRCg6a)")
print("="*60)
print(f"\n序列长度: {CONFIG['seq_length']:,} bp (65k)")
print(f"滑动步长: {CONFIG['stride']:,} bp (50% overlap)")
print(f"最大 N 比例: {CONFIG['max_n_ratio']*100}%")
print(f"验证集比例: {CONFIG['val_ratio']*100}%")

Chicken Caduceus 预训练数据准备 (GRCg6a)

序列长度: 65,536 bp (65k)
滑动步长: 32,768 bp (50% overlap)
最大 N 比例: 5.0%
验证集比例: 2.0%


In [19]:
# Cell 3: 下载 GRCg6a 鸡基因组
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

genome_gz = DATA_DIR / 'GRCg6a_genome.fa.gz'

if not genome_gz.exists():
    print("下载鸡基因组 GRCg6a (galGal6)...")
    print("预计大小: ~350 MB, 需要几分钟\n")
    try:
        with DownloadProgressBar(unit='B', unit_scale=True, desc="Ensembl GRCg6a") as t:
            urllib.request.urlretrieve(GENOME_URL, filename=genome_gz, reporthook=t.update_to)
        print(f"\n下载完成! 大小: {os.path.getsize(genome_gz)/1e6:.1f} MB")
    except Exception as e:
        print(f"Ensembl 下载失败: {e}")
        print("尝试 NCBI 备用链接...\n")
        try:
            with DownloadProgressBar(unit='B', unit_scale=True, desc="NCBI GRCg6a") as t:
                urllib.request.urlretrieve(GENOME_URL_BACKUP, filename=genome_gz, reporthook=t.update_to)
            print(f"\n下载完成! 大小: {os.path.getsize(genome_gz)/1e6:.1f} MB")
        except Exception as e2:
            print(f"NCBI 下载也失败: {e2}")
            print("\n请手动下载基因组文件并上传到 /content/chicken_genome/GRCg6a_genome.fa.gz")
else:
    print(f"基因组文件已存在: {os.path.getsize(genome_gz)/1e6:.1f} MB")

基因组文件已存在: 320.3 MB


In [20]:
# Cell 4: 解压基因组
genome_fa = DATA_DIR / 'GRCg6a_genome.fa'

if not genome_fa.exists():
    print("解压基因组文件 (需要 1-2 分钟)...")
    with gzip.open(genome_gz, 'rb') as f_in:
        with open(genome_fa, 'wb') as f_out:
            # 分块读取，显示进度
            total_size = os.path.getsize(genome_gz)
            with tqdm(total=total_size, unit='B', unit_scale=True, desc="解压") as pbar:
                while True:
                    chunk = f_in.read(1024*1024)  # 1MB chunks
                    if not chunk:
                        break
                    f_out.write(chunk)
                    pbar.update(len(chunk))
    print(f"\n解压完成! 大小: {os.path.getsize(genome_fa)/1e9:.2f} GB")
else:
    print(f"解压文件已存在: {os.path.getsize(genome_fa)/1e9:.2f} GB")

解压文件已存在: 1.08 GB


In [21]:
# Cell 5: 分析基因组并加载染色体序列
print("加载和分析基因组...\n")

# 存储染色体序列
chromosomes = {}
gnome_stats = {
    'total_sequences': 0,
    'total_length': 0,
    'main_chromosomes': 0,
    'main_length': 0,
    'gc_count': 0,
    'n_count': 0,
}

with open(genome_fa, 'r') as f:
    for record in tqdm(SeqIO.parse(f, 'fasta'), desc="读取染色体"):
        genome_stats['total_sequences'] += 1
        seq = str(record.seq).upper()
        seq_len = len(seq)
        genome_stats['total_length'] += seq_len

        # 提取染色体名称 (处理不同格式)
        chrom_id = record.id.split()[0]
        # 移除可能的前缀
        if chrom_id.startswith('chr'):
            chrom_name = chrom_id[3:]
        else:
            chrom_name = chrom_id

        # 检查是否是主要染色体
        is_main = chrom_name in CONFIG['main_chromosomes']

        if is_main or not CONFIG['use_main_chromosomes']:
            chromosomes[chrom_name] = seq
            genome_stats['main_chromosomes'] += 1
            genome_stats['main_length'] += seq_len
            genome_stats['gc_count'] += seq.count('G') + seq.count('C')
            genome_stats['n_count'] += seq.count('N')

# 计算统计信息
gc_content = genome_stats['gc_count'] / genome_stats['main_length'] * 100 if genome_stats['main_length'] > 0 else 0
n_content = genome_stats['n_count'] / genome_stats['main_length'] * 100 if genome_stats['main_length'] > 0 else 0

print(f"\n" + "="*50)
print("GRCg6a 基因组统计")
print("="*50)
print(f"总序列数: {genome_stats['total_sequences']}")
print(f"总长度: {genome_stats['total_length']:,} bp ({genome_stats['total_length']/1e9:.2f} Gb)")
print(f"\n主要染色体数: {genome_stats['main_chromosomes']}")
print(f"主要染色体长度: {genome_stats['main_length']:,} bp ({genome_stats['main_length']/1e9:.2f} Gb)")
print(f"GC 含量: {gc_content:.2f}%")
print(f"N 含量: {n_content:.2f}%")

print(f"\n各染色体长度:")
for chrom in sorted(chromosomes.keys(), key=lambda x: (not x.isdigit(), int(x) if x.isdigit() else x)):
    print(f"  chr{chrom}: {len(chromosomes[chrom])/1e6:.1f} Mb")

加载和分析基因组...



读取染色体: 464it [00:25, 17.90it/s]


GRCg6a 基因组统计
总序列数: 1392
总长度: 3,196,096,275 bp (3.20 Gb)

主要染色体数: 105
主要染色体长度: 3,150,469,794 bp (3.15 Gb)
GC 含量: 41.70%
N 含量: 0.93%

各染色体长度:
  chr1: 197.6 Mb
  chr2: 149.7 Mb
  chr3: 110.8 Mb
  chr4: 91.3 Mb
  chr5: 59.8 Mb
  chr6: 36.4 Mb
  chr7: 36.7 Mb
  chr8: 30.2 Mb
  chr9: 24.2 Mb
  chr10: 21.1 Mb
  chr11: 20.2 Mb
  chr12: 20.4 Mb
  chr13: 19.2 Mb
  chr14: 16.2 Mb
  chr15: 13.1 Mb
  chr16: 2.8 Mb
  chr17: 10.8 Mb
  chr18: 11.4 Mb
  chr19: 10.3 Mb
  chr20: 13.9 Mb
  chr21: 6.8 Mb
  chr22: 5.5 Mb
  chr23: 6.1 Mb
  chr24: 6.5 Mb
  chr25: 4.0 Mb
  chr26: 6.1 Mb
  chr27: 8.1 Mb
  chr28: 5.1 Mb
  chr30: 1.8 Mb
  chr31: 6.2 Mb
  chr32: 0.7 Mb
  chr33: 7.8 Mb
  chrMT: 0.0 Mb
  chrW: 6.8 Mb
  chrZ: 82.5 Mb


In [22]:
# Cell 6: 生成 65k 滑动窗口序列
print("生成 65k bp 滑动窗口序列...")
print(f"窗口大小: {CONFIG['seq_length']:,} bp")
print(f"滑动步长: {CONFIG['stride']:,} bp")
print(f"最大 N 比例: {CONFIG['max_n_ratio']*100}%\n")

sequences = []
stats = {
    'total_windows': 0,
    'filtered_n': 0,
    'filtered_gc': 0,
    'kept': 0,
}

for chrom_name, chrom_seq in tqdm(chromosomes.items(), desc="处理染色体"):
    chrom_len = len(chrom_seq)

    # 跳过太短的染色体
    if chrom_len < CONFIG['seq_length']:
        continue

    # 滑动窗口
    for start in range(0, chrom_len - CONFIG['seq_length'] + 1, CONFIG['stride']):
        stats['total_windows'] += 1
        end = start + CONFIG['seq_length']
        window_seq = chrom_seq[start:end]

        # 标准化序列 (非ACGT替换为N)
        window_seq = ''.join(c if c in 'ACGT' else 'N' for c in window_seq)

        # 过滤 N 比例过高的窗口
        n_ratio = window_seq.count('N') / len(window_seq)
        if n_ratio > CONFIG['max_n_ratio']:
            stats['filtered_n'] += 1
            continue

        # 过滤 GC 含量异常的窗口
        gc_count = window_seq.count('G') + window_seq.count('C')
        gc_ratio = gc_count / len(window_seq)
        if gc_ratio < CONFIG['min_gc_ratio'] or gc_ratio > CONFIG['max_gc_ratio']:
            stats['filtered_gc'] += 1
            continue

        # 保存序列
        sequences.append({
            'id': f"chr{chrom_name}_{start}_{end}",
            'seq': window_seq,
            'chrom': chrom_name,
            'start': start,
            'end': end,
            'gc_ratio': gc_ratio,
            'n_ratio': n_ratio,
        })
        stats['kept'] += 1

print(f"\n" + "="*50)
print("序列生成统计")
print("="*50)
print(f"总窗口数: {stats['total_windows']:,}")
print(f"过滤 (N>{CONFIG['max_n_ratio']*100}%): {stats['filtered_n']:,}")
print(f"过滤 (GC异常): {stats['filtered_gc']:,}")
print(f"保留序列: {stats['kept']:,}")
print(f"\n总数据量: {stats['kept'] * CONFIG['seq_length'] / 1e9:.2f} Gb")
print(f"基因组覆盖倍数: {stats['kept'] * CONFIG['seq_length'] / genome_stats['main_length']:.1f}x")

生成 65k bp 滑动窗口序列...
窗口大小: 65,536 bp
滑动步长: 32,768 bp
最大 N 比例: 5.0%



处理染色体: 100%|██████████| 35/35 [03:30<00:00,  6.03s/it]


序列生成统计
总窗口数: 31,997
过滤 (N>5.0%): 351
过滤 (GC异常): 5
保留序列: 31,641

总数据量: 2.07 Gb
基因组覆盖倍数: 0.7x


In [23]:
# Cell 7: 按染色体分割数据集 (避免数据泄露)
print("按染色体分割训练/验证/测试集...")
print("(使用染色体级别分割，避免数据泄露)\n")

random.seed(CONFIG['seed'])

# 按染色体分组
chrom_to_seqs = defaultdict(list)
for seq in sequences:
    chrom_to_seqs[seq['chrom']].append(seq)

# 染色体列表 (按大小排序)
chrom_list = sorted(chrom_to_seqs.keys(),
                    key=lambda x: len(chrom_to_seqs[x]),
                    reverse=True)

# 分配染色体到不同数据集
# 测试集: chr1 (最大的染色体，用于最终评估)
# 验证集: chr2 (第二大，用于训练监控)
# 训练集: 其他所有染色体
test_chroms = ['1']
val_chroms = ['2']
train_chroms = [c for c in chrom_list if c not in test_chroms + val_chroms]

train_seqs = []
val_seqs = []
test_seqs = []

for chrom in train_chroms:
    train_seqs.extend(chrom_to_seqs[chrom])
for chrom in val_chroms:
    val_seqs.extend(chrom_to_seqs[chrom])
for chrom in test_chroms:
    test_seqs.extend(chrom_to_seqs[chrom])

# 打乱训练集
random.shuffle(train_seqs)

print(f"数据集分割:")
print(f"  训练集: {len(train_seqs):,} 序列 ({len(train_seqs)/len(sequences)*100:.1f}%)")
print(f"    染色体: {', '.join(sorted(train_chroms, key=lambda x: (not x.isdigit(), int(x) if x.isdigit() else ord(x))))}")
print(f"  验证集: {len(val_seqs):,} 序列 ({len(val_seqs)/len(sequences)*100:.1f}%)")
print(f"    染色体: {', '.join(val_chroms)}")
print(f"  测试集: {len(test_seqs):,} 序列 ({len(test_seqs)/len(sequences)*100:.1f}%)")
print(f"    染色体: {', '.join(test_chroms)}")

按染色体分割训练/验证/测试集...
(使用染色体级别分割，避免数据泄露)

数据集分割:
  训练集: 21,082 序列 (66.6%)
    染色体: 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, W, Z
  验证集: 4,549 序列 (14.4%)
    染色体: 2
  测试集: 6,010 序列 (19.0%)
    染色体: 1


In [24]:
# Cell 8: 创建 HDF5 数据集 (高效存储)
print("创建 HDF5 数据集...")

# DNA 编码映射
# Caduceus 原生编码 (vocab_size=16)
# [MASK]=3, [PAD]=4, A=7, C=8, G=9, T=10, N=11
DNA_VOCAB = {'A': 7, 'C': 8, 'G': 9, 'T': 10, 'N': 11}

def encode_sequence(seq):
    """将 DNA 序列编码为整数数组"""
    return np.array([DNA_VOCAB.get(c, 11) for c in seq], dtype=np.uint8)

def create_hdf5_dataset(seqs, output_path, desc=""):
    """创建 HDF5 数据集"""
    with h5py.File(output_path, 'w') as f:
        # 创建数据集
        n_seqs = len(seqs)
        seq_len = CONFIG['seq_length']

        # 序列数据 (uint8 节省空间)
        sequences_ds = f.create_dataset(
            'sequences',
            shape=(n_seqs, seq_len),
            dtype=np.uint8,
            chunks=(min(100, n_seqs), seq_len),
            compression='gzip',
            compression_opts=4
        )

        # 元数据
        ids = []
        chroms = []
        starts = []
        ends = []
        gc_ratios = []

        # 填充数据
        for i, seq_info in enumerate(tqdm(seqs, desc=desc)):
            sequences_ds[i] = encode_sequence(seq_info['seq'])
            ids.append(seq_info['id'])
            chroms.append(seq_info['chrom'])
            starts.append(seq_info['start'])
            ends.append(seq_info['end'])
            gc_ratios.append(seq_info['gc_ratio'])

        # 保存元数据
        f.create_dataset('ids', data=np.array(ids, dtype='S50'))
        f.create_dataset('chroms', data=np.array(chroms, dtype='S10'))
        f.create_dataset('starts', data=np.array(starts, dtype=np.int64))
        f.create_dataset('ends', data=np.array(ends, dtype=np.int64))
        f.create_dataset('gc_ratios', data=np.array(gc_ratios, dtype=np.float32))

        # 保存配置
        f.attrs['seq_length'] = seq_len
        f.attrs['n_sequences'] = n_seqs
        f.attrs['genome_version'] = 'GRCg6a'
        f.attrs['vocab'] = json.dumps(DNA_VOCAB)

    return output_path

# 创建数据集
train_h5 = PROCESSED_DIR / 'train_65k.h5'
val_h5 = PROCESSED_DIR / 'val_65k.h5'
test_h5 = PROCESSED_DIR / 'test_65k.h5'

print(f"\n创建训练集 ({len(train_seqs):,} 序列)...")
create_hdf5_dataset(train_seqs, train_h5, desc="训练集")
print(f"  保存到: {train_h5}")
print(f"  大小: {os.path.getsize(train_h5)/1e9:.2f} GB")

print(f"\n创建验证集 ({len(val_seqs):,} 序列)...")
create_hdf5_dataset(val_seqs, val_h5, desc="验证集")
print(f"  保存到: {val_h5}")
print(f"  大小: {os.path.getsize(val_h5)/1e6:.1f} MB")

print(f"\n创建测试集 ({len(test_seqs):,} 序列)...")
create_hdf5_dataset(test_seqs, test_h5, desc="测试集")
print(f"  保存到: {test_h5}")
print(f"  大小: {os.path.getsize(test_h5)/1e6:.1f} MB")

创建 HDF5 数据集...

创建训练集 (21,082 序列)...


训练集: 100%|██████████| 21082/21082 [53:49<00:00,  6.53it/s]


  保存到: /content/processed/train_65k.h5
  大小: 0.41 GB

创建验证集 (4,549 序列)...


验证集: 100%|██████████| 4549/4549 [11:37<00:00,  6.53it/s]


  保存到: /content/processed/val_65k.h5
  大小: 89.4 MB

创建测试集 (6,010 序列)...


测试集: 100%|██████████| 6010/6010 [15:19<00:00,  6.54it/s]

  保存到: /content/processed/test_65k.h5
  大小: 117.9 MB


In [26]:
# Cell 9: 同时保存 FASTA 格式 (备用)
print("保存 FASTA 格式数据 (备用)...")

def save_fasta(seqs, path, desc=""):
    """保存序列为 FASTA 格式"""
    records = []
    for s in tqdm(seqs, desc=desc):
        record = SeqRecord(
            Seq(s['seq']),
            id=s['id'],
            description=f"chrom={s['chrom']} start={s['start']} end={s['end']} gc={s['gc_ratio']:.3f}"
        )
        records.append(record)

    with gzip.open(path, 'wt') as f:
        SeqIO.write(records, f, 'fasta')
    return path

train_fa = PROCESSED_DIR / 'train_65k.fa.gz'
val_fa = PROCESSED_DIR / 'val_65k.fa.gz'

save_fasta(train_seqs, train_fa, desc="训练集 FASTA")
print(f"训练集 FASTA: {os.path.getsize(train_fa)/1e6:.1f} MB")

save_fasta(val_seqs, val_fa, desc="验证集 FASTA")
print(f"验证集 FASTA: {os.path.getsize(val_fa)/1e6:.1f} MB")

保存 FASTA 格式数据 (备用)...


训练集 FASTA: 100%|██████████| 21082/21082 [00:04<00:00, 4553.84it/s]


KeyboardInterrupt: 

In [29]:
# Cell 10: 保存配置和统计信息
print("保存配置和统计信息...")

# 数据集统计
dataset_info = {
    'genome_version': 'GRCg6a',
    'genome_source': 'Ensembl release-104',
    'seq_length': CONFIG['seq_length'],
    'stride': CONFIG['stride'],
    'train_sequences': len(train_seqs),
    'val_sequences': len(val_seqs),
    'test_sequences': len(test_seqs),
    'total_sequences': len(sequences),
    'train_chromosomes': sorted(train_chroms, key=lambda x: (not x.isdigit(), int(x) if x.isdigit() else ord(x))),
    'val_chromosomes': val_chroms,
    'test_chromosomes': test_chroms,
    'genome_stats': genome_stats,
    'preprocessing_config': CONFIG,
    'dna_vocab': DNA_VOCAB,
}

# 保存为 JSON
config_path = PROCESSED_DIR / 'dataset_info.json'
with open(config_path, 'w') as f:
    json.dump(dataset_info, f, indent=2)
print(f"配置保存到: {config_path}")

# 打印摘要
print(f"\n" + "="*60)
print("数据集摘要")
print("="*60)
print(f"基因组版本: GRCg6a (galGal6)")
print(f"序列长度: {CONFIG['seq_length']:,} bp")
print(f"训练序列: {len(train_seqs):,}")
print(f"验证序列: {len(val_seqs):,}")
print(f"测试序列: {len(test_seqs):,}")
print(f"总数据量: {len(sequences) * CONFIG['seq_length'] / 1e9:.2f} Gb")

保存配置和统计信息...
配置保存到: /content/processed/dataset_info.json

数据集摘要
基因组版本: GRCg6a (galGal6)
序列长度: 65,536 bp
训练序列: 21,082
验证序列: 4,549
测试序列: 6,010
总数据量: 2.07 Gb


In [30]:
# Cell 11: 验证 HDF5 数据集
print("验证 HDF5 数据集...")

def verify_hdf5(path):
    """验证 HDF5 数据集"""
    with h5py.File(path, 'r') as f:
        print(f"\n文件: {path.name}")
        print(f"  序列数: {f.attrs['n_sequences']}")
        print(f"  序列长度: {f.attrs['seq_length']}")
        print(f"  基因组版本: {f.attrs['genome_version']}")

        # 检查数据形状
        seqs = f['sequences']
        print(f"  数据形状: {seqs.shape}")
        print(f"  数据类型: {seqs.dtype}")

        # 抽样检查
        sample_idx = random.randint(0, len(seqs)-1)
        sample_seq = seqs[sample_idx]
        sample_id = f['ids'][sample_idx].decode()

        # 解码序列
        vocab_inv = {7: 'A', 8: 'C', 9: 'G', 10: 'T', 11: 'N'}
        decoded = ''.join(vocab_inv[x] for x in sample_seq[:50])

        print(f"  样本 ID: {sample_id}")
        print(f"  样本序列 (前50bp): {decoded}...")

        # 统计碱基分布
        unique, counts = np.unique(sample_seq, return_counts=True)
        print(f"  碱基分布: ", end="")
        for u, c in zip(unique, counts):
            print(f"{vocab_inv[u]}={c/len(sample_seq)*100:.1f}% ", end="")
        print()

verify_hdf5(train_h5)
verify_hdf5(val_h5)
verify_hdf5(test_h5)

print("\n数据验证通过!")

验证 HDF5 数据集...

文件: train_65k.h5
  序列数: 21082
  序列长度: 65536
  基因组版本: GRCg6a
  数据形状: (21082, 65536)
  数据类型: uint8
  样本 ID: chr10_16646144_16711680
  样本序列 (前50bp): TTTTCTCTGAATCCAAGGTGAACAAAATTAAAATTGTTAAAGGGTTAAAA...
  碱基分布: A=29.0% C=19.7% G=20.3% T=31.0% 

文件: val_65k.h5
  序列数: 4549
  序列长度: 65536
  基因组版本: GRCg6a
  数据形状: (4549, 65536)
  数据类型: uint8
  样本 ID: chr2_62160896_62226432
  样本序列 (前50bp): TAAAAAGCGCGTGTCCAAGGTAACAGCTAGCAATACAGGGCTGACCTGTG...
  碱基分布: A=27.2% C=20.1% G=23.0% T=29.7% 

文件: test_65k.h5
  序列数: 6010
  序列长度: 65536
  基因组版本: GRCg6a
  数据形状: (6010, 65536)
  数据类型: uint8
  样本 ID: chr1_158007296_158072832
  样本序列 (前50bp): CTGGAGGACACTGATGGACAGGTCCTGGCAAAGGGTTGTGAAATTTGAGG...
  碱基分布: A=28.9% C=19.7% G=21.0% T=30.4% 

数据验证通过!


In [ ]:
# Cell 12: 打包所有文件
print("打包所有文件...")

output_tar = OUTPUT_DIR / 'chicken_pretrain_data_GRCg6a.tar.gz'

with tarfile.open(output_tar, 'w:gz') as tar:
    # 添加 HDF5 文件
    tar.add(train_h5, arcname='data/train_65k.h5')
    tar.add(val_h5, arcname='data/val_65k.h5')
    tar.add(test_h5, arcname='data/test_65k.h5')

    # 添加 FASTA 文件 (备用)
    tar.add(train_fa, arcname='data/train_65k.fa.gz')
    tar.add(val_fa, arcname='data/val_65k.fa.gz')

    # 添加配置文件
    tar.add(config_path, arcname='data/dataset_info.json')

print(f"\n打包完成!")
print(f"输出文件: {output_tar}")
print(f"文件大小: {os.path.getsize(output_tar)/1e9:.2f} GB")

打包所有文件...


In [28]:
# Cell 13: 下载文件
print("="*60)
print("准备下载")
print("="*60)

from google.colab import files

print(f"\n文件: chicken_pretrain_data_GRCg6a.tar.gz")
print(f"大小: {os.path.getsize(output_tar)/1e9:.2f} GB")
print(f"\n包含内容:")
print(f"  - data/train_65k.h5 (训练集 HDF5)")
print(f"  - data/val_65k.h5 (验证集 HDF5)")
print(f"  - data/test_65k.h5 (测试集 HDF5)")
print(f"  - data/train_65k.fa.gz (训练集 FASTA)")
print(f"  - data/val_65k.fa.gz (验证集 FASTA)")
print(f"  - data/dataset_info.json (配置信息)")

print(f"\n开始下载...")
files.download(str(output_tar))

print(f"\n" + "="*60)
print("下一步操作")
print("="*60)
print(f"1. 将下载的文件上传到 H200 服务器")
print(f"2. 解压: tar -xzf chicken_pretrain_data_GRCg6a.tar.gz")
print(f"3. 运行训练: bash run_pretrain_65k.sh train")

准备下载

文件: chicken_pretrain_data_GRCg6a.tar.gz
大小: 0.88 GB

包含内容:
  - data/train_65k.h5 (训练集 HDF5)
  - data/val_65k.h5 (验证集 HDF5)
  - data/test_65k.h5 (测试集 HDF5)
  - data/train_65k.fa.gz (训练集 FASTA)
  - data/val_65k.fa.gz (验证集 FASTA)
  - data/dataset_info.json (配置信息)

开始下载...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


下一步操作
1. 将下载的文件上传到 H200 服务器
2. 解压: tar -xzf chicken_pretrain_data_GRCg6a.tar.gz
3. 运行训练: bash run_pretrain_65k.sh train


---

## 使用说明

### 在 H200 服务器上解压和使用

```bash
# 1. 上传文件到服务器
scp chicken_pretrain_data_GRCg6a.tar.gz user@server:/path/to/caduceus/

# 2. 解压
cd /path/to/caduceus
tar -xzf chicken_pretrain_data_GRCg6a.tar.gz

# 3. 检查文件
ls -la data/
# 应该看到:
# train_65k.h5
# val_65k.h5  
# test_65k.h5
# train_65k.fa.gz
# val_65k.fa.gz
# dataset_info.json

# 4. 开始训练
bash run_pretrain_65k.sh train
```

### 数据格式说明

**HDF5 文件结构:**
- `sequences`: (N, 65536) uint8 数组，DNA 编码 (A=7, C=8, G=9, T=10, N=11 (Caduceus原生编码))
- `ids`: 序列 ID
- `chroms`: 染色体名称
- `starts`: 起始位置
- `ends`: 结束位置
- `gc_ratios`: GC 含量

**读取示例:**
```python
import h5py
import numpy as np

with h5py.File('data/train_65k.h5', 'r') as f:
    sequences = f['sequences'][:]  # 加载所有序列
    ids = [x.decode() for x in f['ids'][:]]
    
# 解码序列
vocab_inv = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: 'N'}
seq_str = ''.join(vocab_inv[x] for x in sequences[0])
```